<center> <h1>Auto-Completing Models to Uncover Missing Physics:
A Deep Dive Tutorial Into Universal Differential Equations
</h1> </center>  

<center> <h3>Vinicius Viena Santana
    </h3>
</center>  

<center> <h3>Norwegian University of Science and Technology, Department of Chemical Engineering, Idelfonso´s lab </h3></center>

**Background**
- Bachelor and Master's degree in industrial engineering from the Federal University of Bahia 🎓
- Ph.D. at the University of Porto, Portugal (Supervised by Idelfonso Nogueira and Chris Rackauckas) 📚

**About me**

Enthusiastic about coding and the Julia programming language ❤️

**Research**
- Currently a researcher at NTNU, Norway (PSE Group, Idelfonso's lab) 🇳🇴
- Research focus: Scientific Machine Learning and AI applied to industrial and chemical engineering 📈

# Modeling in Natural Sciences

Building simple and powerful models is time consuming and expensive 💰

* Models compete for acceptance. Should we favor rigor or simplicity, exactness or usefulness - the 10 USD or 1000 USD model? 

* Always start by trying the simplest model and then only add complexity to the extent needed. This is the 10 USD approach (Levenspiel, 2002, Modeling in Chemical Engineering).

* How can we do minimal extensions that would allow our **SIMPLE** model to fit the data?

# Universal Differential Equations [Rackauckas et al. (2020)](https://arxiv.org/abs/2001.04385)

~ A small black box into your simple model ~

* The universal approximator aim to reduce mechanistic model non-explained residuals through small mechanistic extensions.


<center><img src="diagram-20231023.svg" width="700"/></center>

# Case Study: UDE in Separation Engineering -- Fixed Bed Adsorption

* What is Fixed Bed Adsorption?

<center><img src="diagram-20231030.svg" width="700"/></center>

* We need conservation equations to describe the transport of chemicals inside of it. 
For columns, writing it down results in Partial Differential Equations.

## The Zero Length Column

* If the column size is sufficiently small, we can use the zero-length column approximation. This simplifies the kinetics estimation, making it easier than with larger column because mass balance is reduced to an ODE instead of a PDE.

$\textrm{Fluid phase mass balance}$

$V_f \frac{dc}{dt} + M_s \frac{dq}{dt} = Q(c_{\text{in}} - c)$

$\textrm{Solid phase mass balance}$

$\frac{dq}{dt} = f(q, q^*)$

$f(q, q^*) = k_1(q^* - q), k_2(q^* - q)^2, k_3(q^{*2} - q^2)/q,...?$

* Why not $f(q, q^*) = \textrm{ANN}(q^*, q,\theta)$

q* is typically expressed as a function of c: $q^* = Q\frac{Kc}{(1 + Kc)}$

# Hands-on UDE implementation

## Loading and Visualizing Data

In [1]:
using Pkg
Pkg.activate() #Activating enviroment
using Plots, LaTeXStrings, DelimitedFiles
plotly() #Plot backend

  Activating project at `C:\Users\Vinic\.julia\environments\v1.9`


Plots.PlotlyBackend()

In [2]:
plot_font = "Computer Modern"
default(fontfamily=plot_font,
        linewidth=2, framestyle=:box, label=nothing, grid=false)

In [46]:
train_data = readdlm("observations.csv", ',');
scatter(train_data[:, 1], train_data[:, 2], xlabel = "time (min)", ylabel = "Concentration (mg/L)", label = "Exp")
plot!(train_data[:, 1], ones(size(train_data, 1))*Cin, xlabel = "time (min)",
        ylabel = "Concentration (mg/L)", label = "Inlet c")

## Defining parameters and Neural Network

In [156]:
begin
# Bed parameters
D_bed = 5*10^-2 #m
L_bed = 5*10^-2 #m
V_bed = 4/3*pi*(D_bed/2)^3
ϵ = 0.45
d_p = 2e-3
ρ_p = 1100 #kg/m3
V_p = 4/3*pi*(d_p/2)^3 
V_solid = V_bed*(1-ϵ)
V_gas = V_bed*ϵ
M_solid = ρ_p*V_solid #kg

# Inlet conditions
Q = 5 #L/min
Cin = 20 # ppm = mg/L

#Isotherm parameters
qmax = 55.54 #mg/g
k = 1.8 #L/mg
kmass = 0.22; #min^-1
end

0.22

### Creating Neural Network using Lux.jl

<center><img src="neural_net.svg" width="600"/></center>

In [157]:
#Scaling parameter (maximum concentration at solid phase)
qstar_max = qmax*k*Cin/(1 + k*Cin);

In [158]:
#Neural Network with 2 inputs and one of this inputs is a function (q*)
using Lux
ɼ = Lux.Chain(x -> x./qstar_max, Lux.Dense(2, 8, tanh), Lux.Dense(8, 10, tanh),
                  Lux.Dense(10, 1)); #Uptake rake

In [159]:
using Random;
rng = Random.default_rng(222)
pinit, st = Lux.setup(rng, ɼ);

In [160]:
ɼ([50; 50], pinit, st)[1]

1-element Vector{Float64}:
 0.28480718564744717

## Defining the UDE model

In [161]:
using DifferentialEquations, DiffEqFlux

In [162]:
using ComponentArrays

In [163]:
function ude(du, u, p, t)
    c = u[1] #Fluid phase concentration
    q = u[2] #Solid phase concentration

    qstar = qmax*k*c/(1.0 + k*c)

    du[1] = Q*(Cin - c)/(V_gas*1e3) - M_solid*1e3*ɼ([qstar q]', p, st)[1][1]/(V_gas*1e3) 
    du[2] = ɼ([qstar q]', p, st)[1][1]
end

ude (generic function with 1 method)

Simulating the UDE before training it - randomly initialized parameters.

In [164]:
Δt = train_data[2, 1] - train_data[1, 1]; #1 min
tinit = train_data[1, 1] #0.0 min
tfinal = train_data[end, 1] #50 min
u0 = [0.0; 0.0];
prob = ODEProblem(ude, u0, (tinit, tfinal), pinit);
#Note that the ODE has two dependent variables but only of them are measured

In [165]:
sol_ude = solve(prob, AutoVern7(KenCarp5(autodiff = true)), abstol = 1e-7, reltol = 1e-7, saveat = Δt);

In [166]:
plot(sol_ude.t, Array(sol_ude)[1, :], label = "untrained UDE", size = (700, 250))
scatter!(train_data[:, 1], train_data[:, 2], xlabel = "time (min)", ylabel = "Concentration (mg/L)", label = "observation")

## Training using only one of the states as training data - c(t)

In [167]:
using SciMLSensitivity, Optimization, OptimizationOptimisers, OptimizationOptimJL, Zygote

In [168]:
function predict(p, states)
    new_prob = remake(prob; p = p)
    sensealg = InterpolatingAdjoint(autojacvec = ReverseDiffVJP(true)) #that is a very important choice to make
    out = Array(solve(new_prob, AutoVern7(KenCarp4(autodiff = false)), abstol = 1e-6, reltol = 1e-6,
    sensealg = sensealg, saveat = Δt))[states, :]; #g = [0, 1] * u
    out
end


predict (generic function with 1 method)

In [169]:
function loss(p)
    y = predict(p, 1) #Call predict function
    loss = sum(abs2, (y .- train_data[:, 2])/Cin) #squared error
    loss
end

loss (generic function with 1 method)

### Setting up optimization problem

In [170]:
loss(pinit) #High initial loss

12.241571739579275

In [171]:
adtype = Optimization.AutoZygote()
optf = Optimization.OptimizationFunction((x,p) -> loss(x), adtype);
optprob = Optimization.OptimizationProblem(optf, ComponentArray(pinit));

In [172]:
losses = Float64[]
callback = function (p, l)
    push!(losses, l)
    if length(losses) % 10 == 0
        println("Current loss after $(length(losses)) iterations: $(losses[end])")
    end
    return false
end


#47 (generic function with 1 method)

In [173]:
@time res1 = Optimization.solve(optprob, ADAM(0.075), callback = callback, maxiters = 270);

Current loss after 10 iterations: 0.9912751016369831
Current loss after 20 iterations: 0.05233927397387695
Current loss after 30 iterations: 0.18011964069645936
Current loss after 40 iterations: 0.08956623192614221
Current loss after 50 iterations: 0.06762473870592468
Current loss after 60 iterations: 0.02934085739176759
Current loss after 70 iterations: 0.022607886716461845
Current loss after 80 iterations: 0.022410285687602102
Current loss after 90 iterations: 0.018872805287614713
Current loss after 100 iterations: 0.01704543106013229
Current loss after 110 iterations: 0.015542540768194571
Current loss after 120 iterations: 0.01423458567782907
Current loss after 130 iterations: 0.013064919704580066
Current loss after 140 iterations: 0.011966802434716766
Current loss after 150 iterations: 0.010987784025897857
Current loss after 160 iterations: 0.010131311805941621
Current loss after 170 iterations: 0.009390651707241132
Current loss after 180 iterations: 0.008742915632061329
Current lo

In [177]:
test_data = readdlm("test_data.csv", ',');

In [186]:
Cin = 20
plot(sol_ude.t, predict(res1.u, 1), label = "Trained UDE", size = (800, 350))
scatter!(train_data[:, 1], train_data[:, 2], xlabel = "time (min)", 
    ylabel = "Concentration (mg/L)", label = "Train data",
legend=:outerbottomright)

In [179]:
Cin = 10.0
test_prob = remake(prob; p = res1.u, tspan = (0.0, 100.), u0 = [0.0, 0.0])
sol_testprob = Array(solve(test_prob, AutoVern7(KenCarp4(autodiff = true)), abstol = 1e-6, reltol = 1e-6,
        saveat = Δt))[1, :];
plot(0.0:1.0:100.0, sol_testprob, label = "Test set UDE", size = (800, 350))
scatter!(test_data[:, 1], test_data[:, 2], xlabel = "time (min)", 
    ylabel = "Concentration (mg/L)", label = "Test data",
legend=:outerbottomright)

## Smoothed Collocation for Fast Two-Stage Training

Training ANNs in ODEs can be slow. Can we speed it up with a warm start?

$ANN(q^*, q) = \frac{dq}{dt}$

How do we estimate dq/dt? First we have to estimate q(t) from c(t).

$V_f \int_{0}^{c}{dc} + M_s \int_{0}^{q}{dq} = Q(c_{\text{in}}t - \int_{0}^{c}{c}dt)$

$q(t) = Q/M_s(c_{\text{in}}t - \int_{0}^{c}{c}dt) - V_f/M_s c(t) $

In [180]:
using DataInterpolations, Integrals

In [187]:
t = train_data[:, 1]
c_exp = train_data[:, 2]
c_itp = AkimaInterpolation(c_exp, t); #Interpolate

In [188]:
integrand(t) = Q/(M_solid*1e3)*c_itp(t)
int_integrand(t) = solve(IntegralProblem((t, p) -> integrand(t), 0.0, t), QuadGKJL()).u
q(t) = Q/(M_solid*1e3)*Cin*t - int_integrand(t) - V_gas/M_solid*c_itp(t)

q (generic function with 1 method)

In [189]:
q_train = q.(t);

We can use Kernel smoothing to estimate dq(t)/dt from q(t)

In [190]:
dq_dt_est, q_smooth = collocate_data(q_train[:, :]', train_data[:, 1], EpanechnikovKernel()); #From DiffEqFlux

([2.535207827004392 2.532016773743905 … 0.010450459894075337 0.008616109173400943], [0.09093161908405922 2.576075130934778 … 53.90842337568725 53.93068327921643])

In [203]:
scatter(t, q_train, xlabel = "time (min)", ylabel = "q (mg/L)", label = "q_exp (estimated)")
plot!(t, q_smooth[:], xlabel = "time (min)",
ylabel = "Concentration (mg/L)", legend=:outerbottomright, label = "Smoothed q", size = (700, 300))

Setting up learning problem

In [192]:
Y = dq_dt_est[:]; #target
X = [qmax*k*c_exp./(1.0 .+ k*c_exp) q_train]'/qstar_max; #Features

2×51 Matrix{Float64}:
  0.176155   0.461967  0.242618  0.234628  …  0.999947  1.00013   0.999656
 -1.5817e-6  0.045919  0.091926  0.138309     0.997175  0.997077  0.997144

In [193]:
ɼ_fast = Lux.Chain(Lux.Dense(2, 8, tanh), Lux.Dense(8, 10, tanh),
                  Lux.Dense(10, 1)); #Uptake rake

In [194]:
using Random;
rng = Random.default_rng(222)
pinit_fast, st_fast = Lux.setup(rng, ɼ_fast);

In [195]:
# Predicting and loss function
function predict_fast(p)
        Y_hat = ɼ_fast(X, p, st_fast)[1][:]
        Y_hat
end

function loss_fast(p)
         Y_hat = predict_fast(p)
         loss = sum(abs2, (Y_hat .- Y))
         return loss
end

loss_fast (generic function with 1 method)

In [196]:
loss_fast(pinit_fast) #testing initial loss

132.19156537433574

Continuing

In [197]:
adtype = Optimization.AutoZygote();
optf_fast = Optimization.OptimizationFunction((x,p) -> loss_fast(x), adtype);
optprob_fast = Optimization.OptimizationProblem(optf_fast, ComponentArray(pinit_fast));

In [198]:
losses_fast = []
callback_fast = function (p, l)
    push!(losses_fast, l)
    if length(losses_fast) % 100 == 0
        println("Current loss after $(length(losses_fast)) iterations: $(losses_fast[end])")
    end
    return false
end

#55 (generic function with 1 method)

In [199]:
@time res_fast = Optimization.solve(optprob_fast, ADAM(1e-2), callback = callback_fast, maxiters = 500);

Current loss after 100 iterations: 4.378522721813006
Current loss after 200 iterations: 1.6674762088658721
Current loss after 300 iterations: 0.5836468296748255
Current loss after 400 iterations: 0.33954833406485174
Current loss after 500 iterations: 0.23219823413579016
 14.679808 seconds (32.51 M allocations: 1.927 GiB, 5.03% gc time, 98.57% compilation time)


In [200]:
p_modified = copy(ComponentArray(pinit))
p_modified.layer_2 = res_fast.u.layer_1
p_modified.layer_3 = res_fast.u.layer_2
p_modified.layer_4 = res_fast.u.layer_3

ComponentVector{Float32,SubArray...}(weight = Float32[-0.3489829 -0.11738045 … -0.38437322 0.68859684], bias = Float32[0.009310756;;])

In [202]:
plot(t, predict(p_modified, 1), label = "First Stage UDE", size = (800, 450))
scatter!(train_data[:, 1], train_data[:, 2], xlabel = "time (min)",  ylabel = "Concentration (mg/L)", label = "observation", legend=:outerbottomright)

## Conclusions


* The Universal Differential Equation (UDE) framework serves as a powerful tool for enhancing the capabilities of basic mathematical models, allowing them to better fit data.
* We explored a straightforward implementation of UDE for estimating uptake kinetics in adsorption processes.
* Additionally, we delved into the application of a fast, two-stage training methodology that employs Kernel smoothing techniques to warm up our ANN parameters.


## Thank you!

Feel free to contact me if you want to know more about it:
    
Email: vinicius.santana@ntnu.no

Code and files are available at: